In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import teamplayeronoffsummary

# fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"

In [ ]:
dfa = []
# for year in range(2022,2010,-1):
for year in [2022]:    
    df11 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Adv_On_{year}.parquet")
    df12 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Adv_Off_{year}.parquet")    
    df2 = pd.merge(df11,df12,on=['GROUP_SET', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME',
            'VS_PLAYER_ID', 'VS_PLAYER_NAME'])
    # df2["Segment"] = "Regular Season"
    df2["Net"] = df2["NET_RATING_x"] - df2["NET_RATING_y"]
    df2["Off"] = df2["OFF_RATING_x"] - df2["OFF_RATING_y"]
    df2["Def"] = df2["DEF_RATING_x"] - df2["DEF_RATING_y"]
    # df2 = df2[['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME','VS_PLAYER_ID',
    #                 'VS_PLAYER_NAME',"GP_x","GP_y","MIN_x","Min_y","Net","Off","Def"]]
    df31 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Adv_On_{year}_PS.parquet")
    df32 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Adv_Off_{year}_PS.parquet")
    df4 = pd.merge(df31,df32,on=['GROUP_SET', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME',
            'VS_PLAYER_ID', 'VS_PLAYER_NAME'])
    df4["Net"] = df4["NET_RATING_x"] - df4["NET_RATING_y"]
    df4["Off"] = df4["OFF_RATING_x"] - df4["OFF_RATING_y"]
    df4["Def"] = df4["DEF_RATING_x"] - df4["DEF_RATING_y"]
    # df4 = df4[['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME','VS_PLAYER_ID',
    #                 'VS_PLAYER_NAME',"GP_x","GP_y","Net","Off","Def"]]
    df5 = pd.merge(df2,df4,on=['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME',
            'VS_PLAYER_ID', 'VS_PLAYER_NAME',])
    df5["Season"]=year + 1
    dfa.append(df5)
df6 = pd.concat(dfa)
df6["image"]=img_DIR_P + df6["VS_PLAYER_ID"].astype(str) + ".png"

In [ ]:
df6.sort_values("VS_PLAYER_ID")[["VS_PLAYER_ID","Net_x"]]

In [ ]:
df7 = df6.query("MIN_x_x > 200 & MIN_x_y > 50")
# df7 =df6.copy()
df7 = df7.nlargest(25,"Net_y")

In [ ]:
xvar = "Net_x"
yvar = "Net_y"
fig = px.scatter(df7, x=xvar, y=yvar,width=800, height=600, hover_name="VS_PLAYER_NAME",hover_data=["Season","Net_x","Net_y","MIN_x_x","MIN_x_y",],color_discrete_sequence =["red"])
# fig_line = px.line(df, x = "RAPM", y = "RAPM",color_discrete_sequence=["black"])
# fig = go.Figure(data = fig_10.data + fig_line.data)
fig.update_layout( width=800, height=600, showlegend=True)
fig.update_layout( title=dict(
                        text="Regular Season vs Playoffs: Offensive Rating",
                        xanchor="auto",
                        yanchor="middle",
                        font=dict(family="Arial, Tahoma, Helvetica", size=25, color="Black")
                    ),
            )
fig.update_xaxes(title_text=xvar)
fig.update_yaxes(title_text=yvar)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

In [ ]:
data = df7.copy()
xvar = "Net_x"
yvar = "Net_y"
adjust_text_dict = {
    # 'expand_points': (0, 0),
    'arrowprops': {
        'arrowstyle': '->',
        'color': 'red'
    }
}
p = (
    ggplot(data,aes(x=xvar,y=yvar,image="image"))
    + geom_point()
    + geom_image(size=0.2)
    + geom_text(aes(y = data[yvar] -0.5,label = "Season"),adjust_text=adjust_text_dict,size=8)
    + theme_idv
    + theme(
        figure_size=(10,8)
    )
)
p

In [ ]:
players = ["LeBron James","Stephen Curry","Kevin Durant","Kawhi Leonard","Giannis Antetokounmpo","Luka Doncic","Nikola Jokic","Jayson Tatum","James Harden","Russell Westbrook"]
pIDs = [get_pID(pp) for pp in players]
df8 = df6[df6["VS_PLAYER_ID"].isin(pIDs)]
df8["image"]=img_DIR_P + df8["VS_PLAYER_ID"].astype(str) + ".png"

In [ ]:
df8

In [ ]:
adjust_text_dict = {
    # 'expand_points': (0, 0),
    'arrowprops': {
        'arrowstyle': '->',
        'color': 'red'
    }
}
p = (
    ggplot(df8,aes(x="Net_x",y="Net_y",image="image"))
    + geom_point()
    + geom_image(size=0.2)
    + geom_text(aes(y = df8["Net_y"] -0.5,label = "Season"),adjust_text=adjust_text_dict,size=6)
    + theme_idv
    + theme(
        figure_size=(10,8)
    )
)
p

In [ ]:
xvar = "Net_x"
yvar = "Net_y"
fig = px.scatter(df8, x=xvar, y=yvar,width=800, height=600, hover_name="VS_PLAYER_NAME",hover_data=["Season","Net_x","Net_y","GP_x_x","GP_x_y"],color_discrete_sequence =["red"])
# fig_line = px.line(df, x = "RAPM", y = "RAPM",color_discrete_sequence=["black"])
# fig = go.Figure(data = fig_10.data + fig_line.data)
fig.update_layout( width=800, height=600, showlegend=True)
fig.update_layout( title=dict(
                        text="Regular Season vs Playoffs: On/Off Net Rating",
                        xanchor="auto",
                        yanchor="middle",
                        font=dict(family="Arial, Tahoma, Helvetica", size=25, color="Black")
                    ),
            )
fig.update_xaxes(title_text=xvar)
fig.update_yaxes(title_text=yvar)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()

## Career On-Offs

In [ ]:
dfa = []
for year in range(2007,2023):  
    df11 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Adv_On_{year}.parquet")
    df12 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Adv_Off_{year}.parquet")    
    df2 = pd.merge(df11,df12,on=['GROUP_SET', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME',
            'VS_PLAYER_ID', 'VS_PLAYER_NAME'])
    df31 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Adv_On_{year}_PS.parquet")
    df32 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Adv_Off_{year}_PS.parquet")
    df4 = pd.merge(df31,df32,on=['GROUP_SET', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME',
            'VS_PLAYER_ID', 'VS_PLAYER_NAME'])
    df5 = pd.merge(df2,df4,on=['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME',
            'VS_PLAYER_ID', 'VS_PLAYER_NAME',])
    df5["Season"]=year + 1
    dfa.append(df5)
    if df5.empty:
        print(year)
df6 = pd.concat(dfa)
# df6["image"]=img_DIR_P + df6["VS_PLAYER_ID"].astype(str) + ".png"

In [ ]:
cols = [
       'VS_PLAYER_ID', 'VS_PLAYER_NAME', 'GP_x_x',
       'MIN_x_x', 'OFF_RATING_x_x', 'DEF_RATING_x_x',
       'NET_RATING_x_x', 'GP_y_x', 'MIN_y_x',
       'OFF_RATING_y_x', 'DEF_RATING_y_x', 'NET_RATING_y_x',
       'GP_x_y', 'MIN_x_y', 'OFF_RATING_x_y', 'DEF_RATING_x_y',
       'NET_RATING_x_y', 'GP_y_y', 'MIN_y_y',
       'OFF_RATING_y_y', 'DEF_RATING_y_y', 'NET_RATING_y_y', 'Season']
df7 = df6[cols]
suff = ["_x_x","_y_x","_x_y","_y_y"]
mul_cols = ["OFF_RATING","DEF_RATING","NET_RATING"]
for s in suff:
    for c in mul_cols:
        df7[f"{c}{s}"] = df6[f"MIN{s}"]*df6[f"{c}{s}"]

In [ ]:
cols2 = ['GP_x_x','MIN_x_x', 'OFF_RATING_x_x', 'DEF_RATING_x_x',
       'NET_RATING_x_x', 'GP_y_x', 'MIN_y_x',
       'OFF_RATING_y_x', 'DEF_RATING_y_x', 'NET_RATING_y_x',
       'GP_x_y', 'MIN_x_y', 'OFF_RATING_x_y', 'DEF_RATING_x_y',
       'NET_RATING_x_y', 'GP_y_y', 'MIN_y_y',
       'OFF_RATING_y_y', 'DEF_RATING_y_y', 'NET_RATING_y_y']

In [ ]:
df8 = df7.groupby(["VS_PLAYER_ID","VS_PLAYER_NAME"])[cols2].sum().reset_index()
for s in suff:
    for c in mul_cols:
        df8[f"{c}{s}"] = round(df8[f"{c}{s}"]/df8[f"MIN{s}"],2)
for s in ["_x","_y"]:    
    df8[f"Net{s}"] = df8[f"NET_RATING_x{s}"] - df8[f"NET_RATING_y{s}"]
    df8[f"Off{s}"] = df8[f"OFF_RATING_x{s}"] - df8[f"OFF_RATING_y{s}"]
    df8[f"Def{s}"] = df8[f"DEF_RATING_x{s}"] - df8[f"DEF_RATING_y{s}"]
df8["image"]=img_DIR_P + df8["VS_PLAYER_ID"].astype(str) + ".png"



In [ ]:
df9 = df8.query("MIN_x_x > 1000 & MIN_x_y > 200")
df9 = df9.nlargest(25,"MIN_x_y")
players = ["LeBron James","Stephen Curry","Kevin Durant","Kawhi Leonard","Giannis Antetokounmpo","Luka Doncic","Nikola Jokic","Jayson Tatum","James Harden","Russell Westbrook","Joel Embiid","Devin Booker","Paul George","Trae Young","Chris Paul","Draymond Green","Jamal Murray","Khris Middleton","Damian Lillard","Jalen Brunson","Donovan Mitchell","Julius Randle","Jimmy Butler","Bam Adebayo","Karl-Anthony Towns","Jaylen Brown","Al Horford","Anthony Davis"]
pIDs = [get_pID(pp) for pp in players]
df9 = df8[df8["VS_PLAYER_ID"].isin(pIDs)]

In [ ]:
data = df9.copy()
xvar = "NET_RATING_x_x"
yvar = "Net_x"
adjust_text_dict = {
    # 'expand_points': (0, 0),
    'arrowprops': {
        'arrowstyle': '->',
        'color': 'red'
    }
}
p = (
    ggplot(data,aes(x=xvar,y=yvar,image="image"))
    + geom_point()
    + geom_hline(yintercept=0)
    + geom_vline(xintercept=0)
    # + geom_abline(intercept = 0, slope = 1, color = "red",size = 1,linetype="--")
    + geom_image(size=0.2)
    + geom_text(aes(y = data[yvar] -1,label = "VS_PLAYER_NAME"),adjust_text=adjust_text_dict,size=8)
    + theme_xkcd(base_size=16)
    # + coord_cartesian(xlim=(-5,15),ylim=(-5,15))
    + labs(
        title="Playoff vs Regular Season Net Rating",
        subtitle="Since 2007-08 Season | Select Star Players | Above Red line better in playoffs",
        x="Regular Season",
        y="Playoffs",
        caption="@SravanNBA | source:nba.com/stats"
    )
    + theme(
        figure_size=(10,8),
        plot_title=element_text(size=24),
        panel_grid_major_x=element_line(linetype="dashed",color="grey",size=0.5),
        panel_grid_major_y=element_line(linetype="dashed",color="grey",size=0.5),
    )
)
p

In [ ]:
data = df9.copy()
xvar = "NET_RATING_x_x"
yvar = "Net_x"
adjust_text_dict = {
    # 'expand_points': (0, 0),
    'arrowprops': {
        'arrowstyle': '->',
        'color': 'red'
    }
}
p = (
    ggplot(data,aes(x=xvar,y=yvar,image="image"))
    + geom_point()
    + geom_hline(yintercept=0)
    + geom_vline(xintercept=0)
    + geom_abline(intercept = 0, slope = 1, color = "red",size = 1,linetype="--")
    + geom_image(size=0.2)
    + geom_text(aes(y = data[yvar] -1,label = "VS_PLAYER_NAME"),adjust_text=adjust_text_dict,size=8)
    + theme_xkcd(base_size=16)
    # + coord_cartesian(xlim=(-5,15),ylim=(-5,15))
    + labs(
        title="Regular Season Net Rating vs On/Off",
        subtitle="Since 2007-08 Season | Select Star Players",
        x="Net Rating",
        y="On/Off Net Rating",
        caption="@SravanNBA | source:nba.com/stats"
    )
    + theme(
        figure_size=(10,8),
        plot_title=element_text(size=24),
        panel_grid_major_x=element_line(linetype="dashed",color="grey",size=0.5),
        panel_grid_major_y=element_line(linetype="dashed",color="grey",size=0.5),
    )
)
p

In [ ]:
data = df9.copy()
xvar = "Net_x"
yvar = "Net_y"
fig = px.scatter(data, x=xvar, y=yvar,width=800, height=600, hover_name="VS_PLAYER_NAME",hover_data=["Net_x","Net_y","MIN_x_x","MIN_x_y"],color_discrete_sequence =["red"])
# fig_line = px.line(df, x = "RAPM", y = "RAPM",color_discrete_sequence=["black"])
# fig = go.Figure(data = fig_10.data + fig_line.data)
fig.update_layout( width=800, height=600, showlegend=True)
fig.update_layout( title=dict(
                        text="Regular Season vs Playoffs: On/Off Net Rating",
                        xanchor="auto",
                        yanchor="middle",
                        font=dict(family="Arial, Tahoma, Helvetica", size=25, color="Black")
                    ),
            )
fig.update_xaxes(title_text=xvar)
fig.update_yaxes(title_text=yvar)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()